In [1]:
import os
import sys
import io
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import geojson
from datetime import datetime
import time
from osgeo import gdal

In [2]:
# Script as function or class?
# Input parameters: timestep t, start and end time (iso format), wX API base url (default Firemap), coordinate point, timedelta (default 24)

In [3]:
t = 0

# Perimeter updates
Historic perimeter updates (2019 and earlier): https://data-nifc.opendata.arcgis.com/datasets/nifc::historic-geomac-perimeters-2019/about

Daily perimeters after 2022: https://nifc.hub.arcgis.com/datasets/nifc::wfigs-daily-perimeters/about

In [4]:
# MARIA 2019 historical perimeter updates
GEOMAC_URL = "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/Historic_GeoMAC_Perimeters_2019/FeatureServer/0/query?"

maria_params = {
    "f": "geojson",          
    "outFields": "active,datecurrent,uniquefireidentifier,fireyear,incidentname,perimeterdatetime,complexparentirwinid,state,localincidentidentifier,shape__Area,shape__Length",
    "where": "incidentname = 'Maria'",
    "returnGeometry": "true",
    "outSR": 4326,
}

GEOMAC_response = requests.get(GEOMAC_URL, params=maria_params)
GEOMAC_json = GEOMAC_response.json()

# Save perimeter updates in GeoDataFrame
perimeters_gdf = gpd.GeoDataFrame.from_features(
    GEOMAC_json["features"],
    crs="EPSG:5070"  # Required CRS for FARSITE input
)

# Save perimeters to GeoJSON
perimeters_gdf.to_file("outputs/maria_perimeters.geojson", driver="GeoJSON")

In [8]:
# BORDER 2 historical perimeter updates
WFIGS_DAILY_URL = "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/WFIGS_Daily_Perimeters_Public/FeatureServer/0/query?"

border2_params = {
    "f": "geojson",          
    "outFields": "attr_IRWINID,attr_FORID,attr_ContainmentDateTime, attr_FireDiscoveryDateTime,attr_CreatedOnDateTime_dt,attr_ModifiedOnDateTime_dt,poly_CreateDate,poly_GISAcres,poly_Acres_AutoCalc,Shape__Area",
    "where": "attr_IncidentName = 'BORDER 2'",
    "returnGeometry": "true",
    "outSR": 4326,
}

WFIGS_response = requests.get(WFIGS_DAILY_URL, params=border2_params)
WFIGS_json = WFIGS_response.json()

# Save perimeter updates in GeoDataFrame
perimeters_gdf = gpd.GeoDataFrame.from_features(
    WFIGS_json["features"],
    crs="EPSG:5070"  # Required CRS for FARSITE input
)

# Save perimeters to GeoJSON
perimeters_gdf.to_file("outputs/border2_perimeters.geojson", driver="GeoJSON")

# Weather observables

In [6]:
# MARIA - Retrieval via InfluxDB Client connection
timedelta = pd.Timedelta(hours=24) 

# Read start time at time step 0 from ignition file
ignition_filepath = "outputs/maria_ignition.geojson"
with open(ignition_filepath) as f:
    gj = json.load(f)
ignition_json = gj["features"][0]

# Define time constraints for t=0
start_time = ignition_json["properties"]["ignition_time"]  # Ignition time
perimeter_update_time = datetime.fromisoformat(start_time) + timedelta
end_time = perimeter_update_time.strftime("%Y-%m-%dT%H:%M:%S")  # 24 hours after ignition time

In [7]:
# BORDER 2 - Retrieval via Firemap/Pylaski API (up to 2 years back)
timedelta = pd.Timedelta(hours=24)


# Read t=0 data from ignition file
ignition_filepath = "outputs/border2_ignition.geojson"
with open(ignition_filepath) as f:
    gj = json.load(f)


# Define time constraints for t=0
ignition_json = gj['features'][0]
start_time = ignition_json["properties"]["ignition_time"]  # Ignition time
perimeter_update_time = datetime.fromisoformat(start_time) + timedelta
end_time = perimeter_update_time.strftime("%Y-%m-%dT%H:%M:%S")  # 24 hours after ignition time


# Read ignition coordinate point
coordinate = ignition_json["geometry"]["coordinates"]
lon, lat = coordinate[0], coordinate[1]

# GET to nearest weather station for wX observables
FIREMAP_WX_URL = "https://firemap.sdsc.edu/pylaski/stations/data?"
timestamp = int(time.time() * 1000)
FIREMAP_WX_PARAMS = {
        'selection': 'closestTo',
        'lat': str(lat),
        'lon': str(lon),
        'observable': ['wind_speed', 'wind_direction'],
        'from': start_time,
        'to': end_time,
        'callback': 'wxData',
        '_': str(timestamp)
    }
FIREMAP_WX_RESPONSE = requests.get(FIREMAP_WX_URL, params=FIREMAP_WX_PARAMS)
print(FIREMAP_WX_RESPONSE.url)


# Save wX observables to JSON
WX_TEXT = FIREMAP_WX_RESPONSE.text.strip()


# Remove callback wrapper if necessary
if WX_TEXT.startswith('wxData(') and WX_TEXT.endswith(')'):
    WX_JSON = WX_TEXT[len('wxData('):-1]
    WX_OBS = json.loads(WX_JSON)
else:
    WX_OBS = FIREMAP_WX_RESPONSE.json()


# Extract wind speed and wind direction observed values
WIND_SPEED_LIST = WX_OBS["features"][0]["properties"]["wind_speed"]
WIND_DIRECTION_LIST = WX_OBS["features"][0]["properties"]["wind_direction"]


# Add wX observations to ignition json
ignition_json["properties"]["wind_speed_list"] = WIND_SPEED_LIST
ignition_json["properties"]["wind_direction_list"] = WIND_DIRECTION_LIST
ignition_json["properties"]["wX_end_time"] = end_time


# Write updated ignition json to ignition file
with open(ignition_filepath, "w") as f:
    json.dump(ignition_json, f, indent=2)

https://firemap.sdsc.edu/pylaski/stations/data?selection=closestTo&lat=32.601&lon=-116.841&observable=wind_speed&observable=wind_direction&from=2025-01-23T13%3A53%3A00&to=2025-01-24T13%3A53%3A00&callback=wxData&_=1770656558674
